<a href="https://colab.research.google.com/github/wtyn/AI-Practice-Tensorflow-Notes/blob/master/Cats_Dogs_Classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python ./cats_dogs_classify.py


W0626 09:52:24.554058 140482110207872 deprecation_wrapper.py:119] From ./cats_dogs_classify.py:34: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.

W0626 09:52:24.554463 140482110207872 deprecation_wrapper.py:119] From ./cats_dogs_classify.py:38: The name tf.parse_single_example is deprecated. Please use tf.io.parse_single_example instead.

Traceback (most recent call last):
  File "./cats_dogs_classify.py", line 158, in <module>
    dataset_test = get_dataset([test_file], batch_size=32, shuffle_num=0)
  File "./cats_dogs_classify.py", line 64, in get_dataset
    dataset = dataset.map(_preprocess_image)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py", line 1772, in map
    MapDataset(self, map_func, preserve_cardinality=False))
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py", line 3197, in __init__
    **flat_structure(self))
  File "/usr/local/lib/python3.6/dist-pac

In [2]:
# 运行此单元格即可装载您的 Google 云端硬盘。
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 数据

In [0]:
import tensorflow as tf
from tensorflow.python.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.python.keras.layers import Flatten, Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.python.keras.models import Model, load_model
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.python.keras.initializers import TruncatedNormal
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

image_size = 224
is_gpu = True

In [0]:

def parse_exmp(serial_exmp):
    # filename_queue = tf.train.string_input_producer(file_list)  # 建立一个队列，其中的参数为tfrecords文件的路径
    # reader = tf.TFRecordReader()  # 实例化读操作，建立读取器
    # _, serialized_example = reader.read(filename_queue)  # 返回文件名和文件

    keys_to_features = {
        'image': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64)
    }

    feats = tf.parse_single_example(serial_exmp, keys_to_features)

    # image = tf.decode_raw(feats['image'], tf.uint8)
    # label = feats['label']

    image = tf.decode_raw(feats['image'], tf.uint8)
    
    label = tf.cast(feats['label'], tf.int64)
    
    return image, label


def _preprocess_image(image, label):
    
    _image = preprocess_input(image)
    
    return _image, label
    


def get_dataset(file_list, shuffle_num=0, batch_size=32):
    
    
    dataset = tf.data.TFRecordDataset(file_list)

    # Maps the parser on every filepath in the array. You can set the number of parallel loaders here
    dataset = dataset.map(parse_exmp)
    
    # 使用lambda函数处理(归一化)
#     dataset = dataset.map(lambda image, label : (preprocess_input(image), label))
#     dataset = dataset.map(lambda image, label: preprocess_image(image, label))
    
#     dataset = dataset.map(_preprocess_image)
    
    
    
    # This dataset will go on forever

    dataset = dataset.repeat()

    # Set the number of datapoints you want to load and shuffle
    if shuffle_num > 0:
        dataset = dataset.shuffle(shuffle_num)

    # Set the batchsize

    dataset = dataset.batch(batch_size)

    # Create an iterator

    iterator = dataset.make_one_shot_iterator()

    # Create your tf representation of the iterator
    
    image, label = iterator.get_next()
    
    
    # Bring your picture back in shape
    
    image = tf.reshape(image, [-1, image_size, image_size, 3])
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    
    # Create a one hot array for your labels
    
    label = tf.one_hot(label, 2)
    
    return image, label



# 数据迭代
def generator(dataset):
    
    next_op = dataset.make_one_shot_iterator().get_next()
    print('----->>')
    with tf.Session() as sess:
        print('----->>')
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
#         coord = tf.train.Coordinator()
#         threads = tf.train.start_queue_runners(coord=coord)
        print('----->>')
        
        x, y = sess.run(next_op)
        return x, y
    
#         while True:
            
#             x, y = sess.run(next_op)
#             #                 x = np.array(x).reshape((-1, size))
#             #                 y = np.array(y).reshape((-1, CLASSIFY_NUM))
#             print('----')
#             x = np.array(x).reshape((-1, image_size, image_size, 3))
# #             x = preprocess_input(x)
#             y = np.array(y).reshape((-1, 2))
            
#             yield (x, y)
# #             coord.request_stop()
# #             coord.join(threads)






In [7]:


# 训练集
train_file = '/content/drive/My Drive/dataset/cats_dogs/cats_dogs_train_224_224.tfrecords'

x_train, y_train = get_dataset([train_file], batch_size=210, shuffle_num=21000)






# 测试集
test_file = '/content/drive/My Drive/dataset/cats_dogs/cats_dogs_test_224_224.tfrecords'

x_test, y_test = get_dataset([test_file], batch_size=32, shuffle_num=0)

# x_test, y_test = dataset_test.make_one_shot_iterator().get_next()

# x_test_all = []
# y_test_all = []


# with tf.Session() as sess:
#     init_op = tf.global_variables_initializer()
#     sess.run(init_op)
#     for i in range(4004//32):
#         image, label = sess.run((x_test, y_test))
#         x_test_all.append(image)
#         y_test_all.append(label)
#         # save_image('logs/%d.png' % i, image[0])

# x_test_all = np.array(x_test_all).reshape((-1, image_size, image_size, 3))

# x_test_all = preprocess_input(x_test_all)

# y_test_all = np.array(y_test_all).reshape((-1, 2))

# print(x_test_all.shape)
# print(y_test_all.shape)

# print(x_test_all[0])



W0626 10:15:03.923094 139955496589184 deprecation.py:323] From <ipython-input-6-db69a3a48ddb>:62: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


# 模型

In [0]:
class Trainsport_resnet50:
    
    @staticmethod
    def build():
        
        # base_model = ResNet50(weights='imagenet', include_top=False, pooling=None,
        #                       input_shape=(img_weight, img_height, color), classes=nb_classes)
        base_model_weights_path = '/content/drive/My Drive/model/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
        base_model = ResNet50(weights='imagenet', include_top=False, 
                              input_shape=(image_size, image_size, 3), 
                              
                              classes=2)

        for layer in base_model.layers:
            layer.trainable = False

        x = base_model.output

        x = Flatten()(x)

#         x = GlobalAveragePooling2D()(x)
        x = Dropout(0.5)(x)
        
        x = Dense(256, activation='relu',
                 kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.01))(x)
        
        x = Dropout(0.5)(x)
        
        predictions = Dense(2, activation='softmax', 
                            kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.01))(x)

        inputs = base_model.input
        outputs = predictions

        model = Model(inputs=inputs, outputs=outputs)
        
#         model.summary()
        
        return model

# 训练

In [7]:

H = None

if is_gpu:
    
    model = Trainsport_resnet50.build()

    model.compile(loss="categorical_crossentropy",  
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
	          metrics=["accuracy"])


    es = EarlyStopping(monitor='val_loss', patience=10)
    mc = ModelCheckpoint('cats_dogs_best_model.h5', monitor='val_loss', save_best_only=True)


    H = model.fit(x_train, y_train, 
          epochs=50, 
          steps_per_epoch=100, 
          validation_data=(x_test, y_test), 
          validation_steps=4004//32, 
          callbacks=[es, mc]
         )
    
   

#     H = model.fit(x_train, y_train,
#                    epochs=100, 
#                    steps_per_epoch=100, 
#                    validation_data=(x_test, y_test),
#                    validation_steps=4004//32,
#                    validation_freq=5
#                   )
 

# model.fit(train_x, train_y, epochs=20, steps_per_epoch=100, validation_data=(test_x, test_y), validation_freq=10)


    




/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
W0626 09:52:38.151105 139784028768128 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:94: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/50
100/100 [==============================] - 126s 1s/step - loss: 0.0922 - acc: 0.9634 - val_loss: 0.0426 - val_acc: 0.9855
Epoch 2/50
100/100 [==============================] - 102s 1s/step - loss: 0.0427 - acc: 0.9843 - val_loss: 0.0417 - val_acc: 0.9865
Epoch 3/50
100/100 [==============================] - 100s 1s/step - loss: 0.0220 - acc: 0.9920 - val_loss: 0.0484 - val_acc: 0.9860
Epoch 4/50
100/100 [==============================] - 100s 1s/step - loss: 0.0144 - acc: 0.9949 - val_loss: 0.0503 - val_acc: 0.9872
Epoch 5/50
100/100 [==============================] - 98s 979ms/step - loss: 0.0096 - acc: 0.9966 - val_loss: 0.0548 - val_acc: 0.9858
Epoch 6/50
100/100 [==============================] - 99s 987ms/step - loss: 0.0076 - acc: 0.9972 - val_loss: 0.0563 - val_acc: 0.9868
Epoch 7/50
100/100 [==============================] - 97s 972ms/step - loss: 0.0054 - acc: 0.9982 - val_loss: 0.0618 - val_acc: 0.9858
Epoch 8/50
100/100 [==============================] - 98s 976ms

KeyboardInterrupt: ignored

# TPU训练

In [0]:
import os



# tpu训练
if not is_gpu:
    
    
    
    TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

    resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER)

    tf.contrib.distribute.initialize_tpu_system(resolver)

    strategy = tf.contrib.distribute.TPUStrategy(resolver)


    with strategy.scope():
        model = Trainsport_resnet50.build()

        model.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
	          metrics=["accuracy"])


# model.fit(dataset_train, 
#           epochs=20, 
#           steps_per_epoch=100, 
#           validation_data=dataset_test, 
#           validation_steps=20, 
#           validation_freq=5
#          )

    model.fit(x_train.astype(np.float32), y_train.astype(np.float32),
          epochs=20, 
          steps_per_epoch=105,
          validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
          validation_freq=5
         )




# 评估

In [8]:
# 保存模型到本地
print("[INFO] 正在保存模型")
model.save('cats_dogs_classify_model.h5')

# f = open('label.bin', "wb")
# f.write(pickle.dumps(labl.bin))
# f.close()

[INFO] 正在保存模型


In [8]:
import matplotlib.pyplot as plt

# 画图
def plot_training(history):
        acc = history.history['acc']
        val_acc = history.history['val_acc']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        epochs = range(len(acc))
        plt.plot(epochs, acc, 'b-')
        plt.plot(epochs, val_acc, 'r')
        plt.title('Training and validation accuracy')
        
        plt.figure()
        plt.plot(epochs, loss, 'b-')
        plt.plot(epochs, val_loss, 'r-')
        plt.title('Training and validation loss')
        plt.show()
        

plot_training(H)


NameError: ignored

In [9]:
x_test_all = []
y_test_all = []


with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for i in range(4004//32):
        image, label = sess.run((x_test, y_test))
        x_test_all.append(image)
        y_test_all.append(label)
        # save_image('logs/%d.png' % i, image[0])

x_test_all = np.array(x_test_all).reshape((-1, image_size, image_size, 3))


y_test_all = np.array(y_test_all).reshape((-1, 2))

print(x_test_all.shape)
print(y_test_all.shape)

print(x_test_all[0])

(4000, 224, 224, 3)
(4000, 2)
[[[  52.060997    -0.7789993  -65.68     ]
  [  47.060997    -6.7789993  -76.68     ]
  [  50.060997    -2.7789993  -84.68     ]
  ...
  [  90.061        7.2210007 -100.68     ]
  [  92.061       15.221001  -103.68     ]
  [  68.061       11.221001   -96.68     ]]

 [[  68.061       17.221      -53.68     ]
  [  48.060997    -3.7789993  -78.68     ]
  [  50.060997    -1.7789993  -86.68     ]
  ...
  [  95.061        9.221001   -88.68     ]
  [  89.061       10.221001   -98.68     ]
  [  66.061        4.2210007  -94.68     ]]

 [[  71.061       21.221      -53.68     ]
  [  46.060997    -4.7789993  -83.68     ]
  [  50.060997    -1.7789993  -87.68     ]
  ...
  [  93.061        7.2210007  -85.68     ]
  [  89.061        8.221001   -93.68     ]
  [  67.061        4.2210007  -90.68     ]]

 ...

 [[ -36.939003   -70.779     -109.68     ]
  [ -37.939003   -70.779     -112.68     ]
  [ -25.939003   -60.779     -103.68     ]
  ...
  [   8.060997   -42.779     -1

In [15]:


# 测试网络模型
def evaluation(model_, x_test, y_test):
    
#     x_test = preprocess_input(x_test)
    
    
    print("[INFO] 正在评估模型")

    predictions = model_.predict(x_test, batch_size=200)

    print(predictions.argmax(axis=1))

    evaluation_score = classification_report(y_test.argmax(axis=1),
                                             predictions.argmax(axis=1), target_names=['cats', 'dogs'])

    print(evaluation_score)

# evaluation(model, x_test, y_test)

print('----->>>>>>')

# 最好模型预测
model_best = Trainsport_resnet50.build()
model_best.load_weights('cats_dogs_best_model.h5')
evaluation(model_best, x_test_all, y_test_all)



----->>>>>>


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


[INFO] 正在评估模型
[0 0 0 ... 1 1 1]
              precision    recall  f1-score   support

        cats       0.99      0.99      0.99      2002
        dogs       0.99      0.99      0.99      1998

    accuracy                           0.99      4000
   macro avg       0.99      0.99      0.99      4000
weighted avg       0.99      0.99      0.99      4000

